# Introduction to torchTT
This section gives an overview of tensor basics and torchTT usage.
For that let
\begin{align}
A \in \mathbb{R}^{n_1 \times \dots \times n_d}
\end{align}
be a tensor of order $d$ and $i$th-mode dimension $n_d$.

For $d = 4$ and $n_i = 2$ it could look like the following.

## Creating a Tensor


In [ ]:
import os
import sys
import torch
import numpy as np
from numpy import size, zeros

_torchtt_path = os.path.join(os.getcwd(), "torchTT")
if _torchtt_path not in sys.path:
    sys.path.insert(0, _torchtt_path)
import torchtt

def frob_norm(t):
    return torch.linalg.norm(t).item()

def tt_datasize(tt):
    return sum(int(c.numel()) for c in tt.cores)

A = torch.rand((2,2,2,2), dtype=torch.float64)
print(A)


As you see, torch prints an order $4$ tensor as nested arrays. Internally tensors are stored in row-major format. This is:
\begin{align}
A(i_1,\dots,i_d) = \sum_{k=1}^d (\prod_{l=k+1}^{d} n_l )i_k
\end{align}
With this you can excess elements directly:


In [ ]:
print(A[0,0,0,0])              # first entry in all dimensions
print(A.flatten()[0])          # first entry in 1D array format
print(A[0,0,1,1])              # Entry (0, 0, 1, 1)


In this sense tensors are just arrays. torchTT adds low-rank tensor formats later.
The two standard creations of tensors are the following:

**all zeros**


In [ ]:
A = torch.zeros((2,2,2), dtype=torch.float64)
print(A)


**identity**

In [ ]:
A = torch.eye(2, dtype=torch.float64)
print(A)
A = torch.eye(4, dtype=torch.float64).reshape(2,2,2,2)
print(A)


## Doing stuff with Tensors

In all above cases, the frobenius Norm
\begin{align}
\| A \|_F = \sqrt{\sum_i A(i_1,\dots,i_d)^2}
\end{align}

can be calculated like this:

In [ ]:
A = torch.rand((2,2,2,2), dtype=torch.float64)
print(frob_norm(A))


Tensors are linear objects, so you can add them (if the dimensions agree) and do scalar multiplication:

In [ ]:
A = torch.rand((2,2,2), dtype=torch.float64)
print(A)
B = torch.rand((2,2,2), dtype=torch.float64)
print(B)
print(A + 2*B)


But most importantly, you can multiply them. For tensor contractions we use Einstein summation via torch.einsum (or torch.tensordot).


In [ ]:
# Tensor index notation is expressed with torch.einsum / torch.tensordot in torch.


Now, if you want to do for example matrix vector multiplication you can do this:

In [ ]:
A = torch.zeros((2,2), dtype=torch.float64)
A[0,1] = 1
A[1,0] = 1
print('A = 
 ' + str(A))
x = torch.tensor([-1.0, 1.0], dtype=torch.float64)
print('x = 
 ' + str(x))
b = A @ x
print('b = 
 ' + str(b))


But it does not stop here. You can do all weird kinds of multiplications and you only need to write Einstein Notation:

In [ ]:
A = torch.ones((2,2,2,2), dtype=torch.float64)
print('A = 
 ' + str(A))
x = torch.zeros((2,2,2), dtype=torch.float64)
x[0,0,0] = -1
x[1,0,0] = 1
x[1,1,0] = 2
print('x = 
 ' + str(x))
b = torch.tensordot(A, x, dims=([1,2,3],[0,1,2]))
print('b = 
 ' + str(b))


Also, you can do a basis transformation 
\begin{align}
\Sigma = P^T A P
\end{align}
like this:

In [ ]:
A = torch.zeros((2,2), dtype=torch.float64)
A[1,0] = 1
A[0,1] = 1
P = torch.zeros((2,2), dtype=torch.float64)
P[0,0] = -1/np.sqrt(2)
P[1,0] = 1/np.sqrt(2)
P[0,1] = 1/np.sqrt(2)
P[1,1] = 1/np.sqrt(2)

S = P @ A @ P.T

print('A = 
' + str(A))
print('P = 
' + str(P))
print('S = 
' + str(S))


As you can see transposing is a not that important concept in tensor mthematic. You just transform each index.

### Doing more complex stuff
You can even do a singular value decomposition of a Tensor just by using the index notation. How you distribute the indices defines what kind of singular value decomposition you do want. I.e.: For $A\in \mathbb{R}^{4 \times 4 \times 3 \times 2} $ an SVD could look like that (depending on the chosen matrification, here $n_1n_2 \times n_3n_4$):
\begin{align}
A(i_1,i_2,i_3,i_4) =  \sum_{k=1}^r U(i_1,i_2,k) S(k,k) V(k,i_3,i_4) = \sum_{k_1,k_2=1}^r U(i_1,i_2,k_1) S(k_1,k_2) V(k_2,i_3,i_4), 
\end{align}
where $r$ is the rank of this matrification and $U$ and $V^t$ have orthogonal columns. This operation is the key to everything what happens on tensors (HOSVD, HSVD etc.). In torch use torch.linalg.svd.


In [ ]:
A = torch.ones((4,4,3,2), dtype=torch.float64)
A_mat = A.reshape(4*4, 3*2)
U, S, Vh = torch.linalg.svd(A_mat, full_matrices=False)
S_mat = torch.diag(S)
U_t = U.reshape(4,4,-1)
V_t = Vh.reshape(-1,3,2)

print('S = 
 '  + str(S_mat))
print('U = 
 '  + str(U_t))
print('V = 
 '  + str(V_t))
A_test = torch.einsum("abk,kl,lcd->abcd", U_t, S_mat, V_t)
print('Test if A can be recovered: ' + str(frob_norm(A - A_test)))


You can also choose the following matrification:
\begin{align}
A(i_1,i_2,i_3,i_4) =  \sum_{k=1}^r U(i_1,k) S(k,k) V(k,i_2,i_3,i_4) = \sum_{k_1,k_2=1}^r U(i_1,k_1) S(k_1,k_2) V(k_2,i_2,i_3,i_4), 
\end{align}
In torch, this translates to:


In [ ]:
A = torch.ones((4,4,3,2), dtype=torch.float64)
A_mat = A.reshape(4, 4*3*2)
U, S, Vh = torch.linalg.svd(A_mat, full_matrices=False)
S_mat = torch.diag(S)
U_t = U.reshape(4,-1)
V_t = Vh.reshape(-1,4,3,2)

print('S = 
 '  + str(S_mat))
print('U = 
 '  + str(U_t))
print('V = 
 '  + str(V_t))
A_test = torch.einsum("ak,kl,libc->aibc", U_t, S_mat, V_t)
print('Test if A can be recovered: ' + str(frob_norm(A - A_test)))


## Tensor Trains (choo choo)

The Tensor Train format is most facinating. It helps you to find a tractable low rank representation (if possible) for a given Tensor. In formulars it looks like:
\begin{align}
A(i_1,\dots,i_d) = \sum_{k_1,\dots,k_{d-1} = 1}^{r_1,\dots,r_{d-1}} A_1(i_1,k_1) A_2(k_1,i_2,k_2)\dots A_{d-1}(k_{d-2},i_{d-1},k_{d-1})  A_d(k_{d-1},i_d) = A_1(i_1)\dots  A_d(i_d)
\end{align}
The last representation is the reason why in physics and elsewhere it is called Matrix Product State (or MPS).
In torchTT you can create a random Tensor Train with 5 'legs' and dimension 3 and ranks 2 like this. The components are order 3 tensors with ranks (2,3,2) except the first and last component. Below you see the components and their mode dimensions.


In [ ]:
A_TT = torchtt.random([3,3,3,3,3], 2)

for i in range(0,5):
    print(A_TT.cores[i])
    print(A_TT.cores[i].shape)


## HT Tensors (here them grow (because they are trees))
HT tensors are not implemented in torchTT. The linearity of tensor trains might not be the best way of capturing the low-rank approximability of a given tensor. What you see below is the result of initializing a random HT tensor and applying the hierarchical SVD in the original tutorial.


In [ ]:
# HTTensor is not supported in torchTT.
A_HT = None
print("HTTensor example skipped (torchTT does not implement HT tensors).")


To see the orthogonality 

In [ ]:
# HTTensor component operations are not supported in torchTT.
print("HTTensor component example skipped.")


Furthermore, the norm of the core (root) tensor is the same as the norm of the whole Tensor.

In [ ]:
# HTTensor norms are not supported in torchTT.
print("HTTensor norm example skipped.")


## A simple ALS implementation on the Tensor Train format
To do linear algebra on tensors formats like tensor train you can use alternating schemes, like ALS (alternating least squared). You iteratively project your big problem on each component solve there and then substitute the component with the 'local' solution. For some reason that works quite well! (It can be shown that the residuum is monotonic decreasing, but that does not imply convergence all the time). 

First we import numpy and specify the number of components of our Tensor train (dim), the size of each 'leg' (n) the maxmal number of ranks of our operator (r) and our solution (r+1), the error magin (eps) and the maximal number of iterations (max_iter).

In [ ]:
import numpy as np
dim = 8
n = 4
r = 2
eps = 10e-8
max_iter = 100


Then we define some indices to create a random symmetric Tensor train operator (A). The multiplication is to make the operator symmetric. Then we cast a random solution and calculate a right hand side (b). In the end we have 
\begin{align}
A x = b, A \in \mathbb{R}^{(n \times \dots \times n) \times (n \times \dots \times n)}, b  \in \mathbb{R}^{n \times \dots \times n}
\end{align}

In [ ]:
dim = 8
n = 4
r = 2
A = torchtt.random([(n, n)] * dim, 2)

solution = torchtt.random([n for _ in range(dim)], r+1)
b = A @ solution

print(torch.sqrt(torchtt.dot(b, b)).item())


Now we initialize a random starting vector $x$. And move the core to the first component with move_core.

In [ ]:
x = torchtt.random([n for _ in range(dim)], r+1)
# torchTT does not expose move_core; use rounding/orthogonalization as needed.
x = x.round(1e-12)


Finally, the ALS iteration is performed. There are max_iter sweeps from left to right. At each sweep we touch each TT component once. We build the projection operators on the component and build the 'local' operator (op) and the local right hand side (rhs). Then, we transform the tensor to a numpy array, reshape it, solve the system of equations and create a new TT core from the solution (sol). This is the new component. Finally we check if the error in the Frobenius norm is smaller than eps. If so, we stop.


In [ ]:
# The ALS micro-iteration shown in the original tutorial is not implemented here.
# Consider torchtt.solvers.amen_solve for TT linear systems instead.
print("ALS micro-iteration example skipped (not implemented in torchTT).")


<div style="text-align: right; color: #a5a9af"> &copy; Michael Götte, Robert Gruhlke, Manuel Marschall, Phillip Trunschke, 2018-2019</div>